# Step1 : Import libraries

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

# Step2 : Working with 3GB data

### 2.1 Download the 3GB data from figshare via API

In [2]:
# API request
article_id = 14226968
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figshareairline_3GB/"
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)
files = data["files"]
files

[{'id': 26844650,
  'name': 'allyears.csv.zip',
  'size': 2405908113,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26844650',
  'supplied_md5': '9e046ac05ecd2c32a256a47dd1098b81',
  'computed_md5': '9e046ac05ecd2c32a256a47dd1098b81'},
 {'id': 26863682,
  'name': 'individual_years.zip',
  'size': 1896206676,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26863682',
  'supplied_md5': '921da748974b07b2a70bbfcc04535a77',
  'computed_md5': '921da748974b07b2a70bbfcc04535a77'},
 {'id': 27515426,
  'name': 'combined_model_data.csv.zip',
  'size': 821308997,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/27515426',
  'supplied_md5': '7638434c44a7d29cbb29fe200b4fd65d',
  'computed_md5': '7638434c44a7d29cbb29fe200b4fd65d'},
 {'id': 27520682,
  'name': 'combined_model_data_parti.parquet.zip',
  'size': 519743915,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/

### 2.2 Extract the zip file

In [3]:
# Create zip file of individual_years.zip

In [4]:
%%time
files_to_dl = ["individual_years.zip"]
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 8.46 s, sys: 28 s, total: 36.5 s
Wall time: 3min 38s


In [5]:
# Extract data into output directory

In [6]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "individual_years.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: user 17.7 s, sys: 1.43 s, total: 19.1 s
Wall time: 19.5 s


### 2.3 Combine CSV files

In [7]:
%%time
## here we are using a normal python way for merging the data 
import pandas as pd
use_cols = ["ArrDelay", "DepDelay", "Distance", "TailNum","UniqueCarrier","Origin","Dest"]
files = glob.glob('figshareairline_3GB/individual_years/*.csv')
df = pd.concat((pd.read_csv(file, index_col=0, usecols=use_cols)
                .assign(year=re.findall("[0-9]+", file)[0])
                for file in files)
              )
df.to_csv("figshareairline_3GB/combined_data.csv")

CPU times: user 2min 45s, sys: 11.6 s, total: 2min 57s
Wall time: 2min 58s


### 2.4 Check occupied space

In [29]:
%%sh
du -sh figshareairline_3GB/combined_data.csv

2.9G	figshareairline_3GB/combined_data.csv


### 2.5 : Perform Simple EDA

In [30]:
%%time
df = pd.read_csv("figshareairline_3GB/combined_data.csv")
print(df["UniqueCarrier"].value_counts())

WN    13194660
DL    10435886
AA     9672922
UA     8821384
US     8286980
NW     6946627
CO     4976761
MQ     3954895
OO     3090853
XE     2350309
HP     2224941
AS     2162672
TW     1890420
EV     1697172
OH     1464176
FL     1265138
YV      854056
B6      811341
DH      693047
9E      521059
F9      336958
HA      274265
TZ      208420
AQ      154381
Name: UniqueCarrier, dtype: int64
CPU times: user 27.8 s, sys: 11.5 s, total: 39.4 s
Wall time: 46.2 s


In [31]:
print(f"Memory usage with float64: {df.memory_usage().sum() / 1e6:.2f} MB")

Memory usage with float64: 5522.52 MB


Summary:
| DataSize | Operating System | RAM | Processor | Is SSD | Space occupied | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|:----------:|
| 3GB    |          MacOS        |  16GB   |    Apple M1 Pro       |   Yes     |     5.52GB     |   47s   |
| 10GB    |          MacOS        |  16GB   |    Apple M1 Pro       |   Yes     |            |      |

# Step3 : Working with 10GB data

### 3.1 Check occupied space

The 10GB Airline data can be found [here.](https://drive.google.com/file/d/1cf88O_KAWy0V2GS-WvysHkd6z0nsOSBL/view)

In [32]:
%%sh
du -sh figshareairline_10GB/combined_10gb.csv

9.3G	figshareairline_10GB/combined_10gb.csv


### 3.2 : Perform Simple EDA

In [33]:
%%time
df = pd.read_csv("figshareairline_10GB/combined_10gb.csv")
print(df["UniqueCarrier"].value_counts())

WN    39583980
DL    31307658
AA    29018766
UA    26464152
US    24860940
NW    20839881
CO    14930283
MQ    11864685
OO     9272559
XE     7050927
HP     6674823
AS     6488016
TW     5671260
EV     5091516
OH     4392528
FL     3795414
YV     2562168
B6     2434023
DH     2079141
9E     1563177
F9     1010874
HA      822795
TZ      625260
AQ      463143
Name: UniqueCarrier, dtype: int64
CPU times: user 1min 20s, sys: 33.1 s, total: 1min 53s
Wall time: 2min 31s


In [34]:
print(f"Memory usage with float64: {df.memory_usage().sum() / 1e6:.2f} MB")

Memory usage with float64: 16567.55 MB


Summary:
| DataSize | Operating System | RAM | Processor | Is SSD | Space occupied | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|:----------:|
| 3GB    |          MacOS        |  16GB   |    Apple M1 Pro       |   Yes     |     5.52GB     |   47s   |
| 10GB    |          MacOS        |  16GB   |    Apple M1 Pro       |   Yes     |    16.57GB     |   2min 31s   |

# Step4 : Tactics to deal with Big data locally

### Approach1 - Select only columns we use

In [35]:
%%time
use_cols = ['UniqueCarrier']
df = pd.read_csv("figshareairline_3GB/combined_data.csv",usecols=use_cols)
print(df["UniqueCarrier"].value_counts())

WN    13194660
DL    10435886
AA     9672922
UA     8821384
US     8286980
NW     6946627
CO     4976761
MQ     3954895
OO     3090853
XE     2350309
HP     2224941
AS     2162672
TW     1890420
EV     1697172
OH     1464176
FL     1265138
YV      854056
B6      811341
DH      693047
9E      521059
F9      336958
HA      274265
TZ      208420
AQ      154381
Name: UniqueCarrier, dtype: int64
CPU times: user 14.2 s, sys: 2.05 s, total: 16.2 s
Wall time: 16.8 s


In [36]:
print(f"Memory usage with float64: {df.memory_usage().sum() / 1e6:.2f} MB")

Memory usage with float64: 690.31 MB


In [37]:
%%time
use_cols = ['UniqueCarrier']
df = pd.read_csv("figshareairline_10GB/combined_10gb.csv",usecols=use_cols)
print(df["UniqueCarrier"].value_counts())

WN    39583980
DL    31307658
AA    29018766
UA    26464152
US    24860940
NW    20839881
CO    14930283
MQ    11864685
OO     9272559
XE     7050927
HP     6674823
AS     6488016
TW     5671260
EV     5091516
OH     4392528
FL     3795414
YV     2562168
B6     2434023
DH     2079141
9E     1563177
F9     1010874
HA      822795
TZ      625260
AQ      463143
Name: UniqueCarrier, dtype: int64
CPU times: user 37.9 s, sys: 2.79 s, total: 40.7 s
Wall time: 41.1 s


In [38]:
print(f"Memory usage with float64: {df.memory_usage().sum() / 1e6:.2f} MB")

Memory usage with float64: 2070.94 MB


### Approach2 - Change data type

In [39]:
%%time
df = pd.read_csv("figshareairline_3GB/combined_data.csv")
print(f"Memory usage with float64: {df.memory_usage().sum() / 1e6:.2f} MB")
print(f"Memory usage with float32: {df.astype('float32', errors='ignore').memory_usage().sum() / 1e6:.2f} MB")
print(df["UniqueCarrier"].value_counts())

Memory usage with float64: 5522.52 MB
Memory usage with float32: 4141.89 MB
WN    13194660
DL    10435886
AA     9672922
UA     8821384
US     8286980
NW     6946627
CO     4976761
MQ     3954895
OO     3090853
XE     2350309
HP     2224941
AS     2162672
TW     1890420
EV     1697172
OH     1464176
FL     1265138
YV      854056
B6      811341
DH      693047
9E      521059
F9      336958
HA      274265
TZ      208420
AQ      154381
Name: UniqueCarrier, dtype: int64
CPU times: user 26.6 s, sys: 9.8 s, total: 36.4 s
Wall time: 41.5 s


In [40]:
%%time
df = pd.read_csv("figshareairline_10GB/combined_10gb.csv")
print(f"Memory usage with float64: {df.memory_usage().sum() / 1e6:.2f} MB")
print(f"Memory usage with float32: {df.astype('float32', errors='ignore').memory_usage().sum() / 1e6:.2f} MB")
print(df["UniqueCarrier"].value_counts())

Memory usage with float64: 16567.55 MB
Memory usage with float32: 12425.66 MB
WN    39583980
DL    31307658
AA    29018766
UA    26464152
US    24860940
NW    20839881
CO    14930283
MQ    11864685
OO     9272559
XE     7050927
HP     6674823
AS     6488016
TW     5671260
EV     5091516
OH     4392528
FL     3795414
YV     2562168
B6     2434023
DH     2079141
9E     1563177
F9     1010874
HA      822795
TZ      625260
AQ      463143
Name: UniqueCarrier, dtype: int64
CPU times: user 1min 20s, sys: 35.9 s, total: 1min 56s
Wall time: 2min 36s


### Approach3 - Load data in chunks

In [41]:
%%time
counts = pd.Series(dtype=int)
for chunk in pd.read_csv("figshareairline_3GB/combined_data.csv", chunksize=10_000_000):
    counts = counts.add(chunk["UniqueCarrier"].value_counts(), fill_value=0)
print(counts.astype(int))

9E      521059
AA     9672922
AQ      154381
AS     2162672
B6      811341
CO     4976761
DH      693047
DL    10435886
EV     1697172
F9      336958
FL     1265138
HA      274265
HP     2224941
MQ     3954895
NW     6946627
OH     1464176
OO     3090853
TW     1890420
TZ      208420
UA     8821384
US     8286980
WN    13194660
XE     2350309
YV      854056
dtype: int64
CPU times: user 26.3 s, sys: 4.19 s, total: 30.5 s
Wall time: 30.8 s


In [42]:
print(f"Memory usage with float64: {chunk.memory_usage().sum() / 1e6:.2f} MB")

Memory usage with float64: 402.52 MB


In [43]:
%%time
counts = pd.Series(dtype=int)
for chunk in pd.read_csv("figshareairline_10GB/combined_10gb.csv", chunksize=10_000_000):
    counts = counts.add(chunk["UniqueCarrier"].value_counts(), fill_value=0)
print(counts.astype(int))

9E     1563177
AA    29018766
AQ      463143
AS     6488016
B6     2434023
CO    14930283
DH     2079141
DL    31307658
EV     5091516
F9     1010874
FL     3795414
HA      822795
HP     6674823
MQ    11864685
NW    20839881
OH     4392528
OO     9272559
TW     5671260
TZ      625260
UA    26464152
US    24860940
WN    39583980
XE     7050927
YV     2562168
dtype: int64
CPU times: user 1min 20s, sys: 12 s, total: 1min 32s
Wall time: 1min 32s


In [44]:
print(f"Memory usage with float64: {chunk.memory_usage().sum() / 1e6:.2f} MB")

Memory usage with float64: 567.55 MB


# Summary

| DataSize | Method | Operating System | RAM | Processor | Is SSD | Space occupied | Time taken |
|:-----------:|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|:----------:|
| 3GB    | Load all data    |    MacOS   |  16GB   | Apple M1 Pro  | Yes | 5.52GB  |   47s      |
| 3GB    | Selected columns |    MacOS   |  16GB   | Apple M1 Pro  | Yes | 0.69GB  |   16.8s    |
| 3GB    | Change data type |    MacOS   |  16GB   | Apple M1 Pro  | Yes | 4.14GB  |   41.5s    |
| 3GB    | Load in chunks   |    MacOS   |  16GB   | Apple M1 Pro  | Yes | 0.40GB  |   30.8s    |
| 10GB   | Load all data    |    MacOS   |  16GB   | Apple M1 Pro  | Yes | 16.57GB |   2min 31s |
| 10GB   | Selected columns |    MacOS   |  16GB   | Apple M1 Pro  | Yes | 2.07GB  |   41.1s    |
| 10GB   | Change data type |    MacOS   |  16GB   | Apple M1 Pro  | Yes | 12.43GB |   2min 36s |
| 10GB   | Load in chunks   |    MacOS   |  16GB   | Apple M1 Pro  | Yes | 0.57GB   |   1min 32s    |